# 第7章: 単語ベクトル
単語の意味を実ベクトルで表現する単語ベクトル（単語埋め込み）に関して，以下の処理を行うプログラムを作成せよ．

## 60. 単語ベクトルの読み込みと表示
Google Newsデータセット（約1,000億単語）での学習済み単語ベクトル（300万単語・フレーズ，300次元）をダウンロードし，”United States”の単語ベクトルを表示せよ．ただし，”United States”は内部的には”United_States”と表現されていることに注意せよ．

In [24]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('files/GoogleNews-vectors-negative300.bin', binary=True)

In [104]:
united_states_vector = model.get_vector('United_States')

## 61. 単語の類似度
“United States”と”U.S.”のコサイン類似度を計算せよ．

In [105]:
us_vector = model.get_vector('U.S.')

In [106]:
def cosine_similarity(a,b):
    return np.dot(a,b)/(np.linalg.norm(a, ord=2)*np.linalg.norm(b, ord=2))


In [107]:
cosine_similarity(united_states_vector,us_vector)

0.7310775

## 62. 類似度の高い単語10件
“United States”とコサイン類似度が高い10語と，その類似度を出力せよ．

In [93]:
similarities = [cosine_similarity(vector,united_states_vector) for vector in model.vectors]

In [101]:
np.array(model.index2word)[np.argsort(similarities)[::-1][:10]]

array(['United_States', 'Unites_States', 'Untied_States', 'United_Sates',
       'U.S.', 'theUnited_States', 'America', 'UnitedStates', 'Europe',
       'countries'], dtype='<U98')

## 63. 加法構成性によるアナロジー
“Spain”の単語ベクトルから”Madrid”のベクトルを引き，”Athens”のベクトルを足したベクトルを計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [109]:
vec = model.get_vector('Spain') -  model.get_vector('Madrid') + model.get_vector('Athens')
similarities2 = [cosine_similarity(vector,vec) for vector in model.vectors]
print(np.array(model.index2word)[np.argsort(similarities2)[::-1][:10]])


array(['Athens', 'Greece', 'Aristeidis_Grigoriadis', 'Ioannis_Drymonakos',
       'Greeks', 'Ioannis_Christou', 'Hrysopiyi_Devetzi', 'Iraklion',
       'Greek', 'Athens_Greece'], dtype='<U98')

In [111]:
print(sorted(similarities2)[::-1][:10])

[0.7528455, 0.66854715, 0.54957783, 0.53614575, 0.5351787, 0.5330226, 0.50884897, 0.5059265, 0.5040616, 0.50341094]


## 64. アナロジーデータでの実験
単語アナロジーの評価データをダウンロードし，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．

In [121]:
from dataclasses import dataclass
@dataclass
class Word:
    word1: str = ""
    word2: str = ''
    word3: str = ''
    word4: str = ''
    @property
    def vec1(self):
        return model.get_vector(self.word1)
    
    @property
    def vec2(self):
        return model.get_vector(self.word2)
    
    @property
    def vec3(self):
        return model.get_vector(self.word3)
    
    @property
    def vec4(self):
        return model.get_vector(self.word4)
    
    def __repr__(self):
        return f'{self.word1} {self.word2} {self.word3} {self.word4}'



In [ ]:
import csv

with open('files/evaluate-words.txt') as f:
    # headerは飛ばす
    f.readline()
    reader = csv.DictReader(f,delimiter=' ',fieldnames=['word1','word2','word3','word4',])
    for row in reader:
        word = Word(row['word1'],row['word2'],row['word3'],row['word4'])
        vec = word.vec2 - word.vec1 + word.vec3
        print(f"{word.word2} - {word.word1} + {word.word3} , {model.similar_by_vector(vec,topn=1)[0]}")


Greece - Athens + Baghdad , ('Baghdad', 0.7489826679229736)
Greece - Athens + Bangkok , ('Bangkok', 0.7431141138076782)
Greece - Athens + Beijing , ('China', 0.7186591625213623)
Greece - Athens + Berlin , ('Germany', 0.6720892786979675)
Greece - Athens + Bern , ('Bern', 0.6902341842651367)
Greece - Athens + Cairo , ('Egypt', 0.7626821398735046)
Greece - Athens + Canberra , ('Canberra', 0.7407206892967224)
Greece - Athens + Hanoi , ('Hanoi', 0.7509901523590088)
Greece - Athens + Havana , ('Havana', 0.7262827754020691)
Greece - Athens + Helsinki , ('Helsinki', 0.7235296964645386)
Greece - Athens + Islamabad , ('Islamabad', 0.7821410894393921)
Greece - Athens + Kabul , ('Kabul', 0.7715259790420532)
Greece - Athens + London , ('London', 0.5891345143318176)
Greece - Athens + Madrid , ('Spain', 0.7145949006080627)
Greece - Athens + Moscow , ('Russia', 0.7455828189849854)
Greece - Athens + Oslo , ('Oslo', 0.7312972545623779)
Greece - Athens + Ottawa , ('Ottawa', 0.6877975463867188)
Greece - A

Egypt - Cairo + Moscow , ('Russia', 0.8410959243774414)
Egypt - Cairo + Oslo , ('Oslo', 0.748042106628418)
Egypt - Cairo + Ottawa , ('Ottawa', 0.7176220417022705)
Egypt - Cairo + Paris , ('France', 0.6924591660499573)
Egypt - Cairo + Rome , ('Italy', 0.6706969738006592)
Egypt - Cairo + Stockholm , ('Sweden', 0.7424593567848206)
Egypt - Cairo + Tehran , ('Iran', 0.8797544240951538)
Egypt - Cairo + Tokyo , ('Japan', 0.7807725071907043)
Egypt - Cairo + Athens , ('Greece', 0.7239887714385986)
Egypt - Cairo + Baghdad , ('Baghdad', 0.7566509246826172)
Egypt - Cairo + Bangkok , ('Thailand', 0.8332600593566895)
Egypt - Cairo + Beijing , ('China', 0.7976171970367432)
Egypt - Cairo + Berlin , ('Germany', 0.6881570816040039)
Egypt - Cairo + Bern , ('Bern', 0.7276052832603455)
Australia - Canberra + Hanoi , ('Hanoi', 0.7349317073822021)
Australia - Canberra + Havana , ('Havana', 0.7401776313781738)
Australia - Canberra + Helsinki , ('Helsinki', 0.7440007328987122)
Australia - Canberra + Islamabad 

## 65. アナロジータスクでの正解率
64の実行結果を用い，意味的アナロジー（semantic analogy）と文法的アナロジー（syntactic analogy）の正解率を測定せよ．

## 66. WordSimilarity-353での評価
The WordSimilarity-353 Test Collectionの評価データをダウンロードし，単語ベクトルにより計算される類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

## 67. k-meansクラスタリング
国名に関する単語ベクトルを抽出し，k-meansクラスタリングをクラスタ数k=5として実行せよ．

## 68. Ward法によるクラスタリング
国名に関する単語ベクトルに対し，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

## 69. t-SNEによる可視化
ベクトル空間上の国名に関する単語ベクトルをt-SNEで可視化せよ．